In [10]:
import fitz  # PyMuPDF
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
import io

def pdf_to_images(pdf_path, dpi=150):
    """将PDF转换为图片，每一页作为一张图片"""
    doc = fitz.open(pdf_path)
    images = []
    
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        
        # 获取图片，dpi越高，图像分辨率越高
        pix = page.get_pixmap(dpi=dpi)
        
        # 将Pixmap转换为PIL Image
        img = Image.open(io.BytesIO(pix.tobytes()))
        images.append((img, pix.width, pix.height))  # 保存图像及其原始宽高
    
    return images

def images_to_ppt(images, ppt_path):
    """将图片列表按比例插入到16:9比例的PPT中"""
    prs = Presentation()
    
    # 设置幻灯片为16:9比例，常见分辨率为1920x1080
    prs.slide_width = Inches(16)
    prs.slide_height = Inches(9)

    slide_width = prs.slide_width  # 幻灯片宽度
    slide_height = prs.slide_height  # 幻灯片高度
    
    for img, img_width, img_height in images:
        slide = prs.slides.add_slide(prs.slide_layouts[5])  # 使用空白幻灯片
        
        # 计算比例，保持图片比例适配PPT页面
        width_ratio = slide_width / img_width
        height_ratio = slide_height / img_height
        scale = min(width_ratio, height_ratio)
        
        # 按比例调整图片大小
        new_width = img_width * scale
        new_height = img_height * scale
        
        # 居中显示图片
        left = (slide_width - new_width) / 2
        top = (slide_height - new_height) / 2
        
        # 保存图片到字节流
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        
        # 插入图片到PPT
        slide.shapes.add_picture(img_byte_arr, left, top, width=new_width, height=new_height)
    
    prs.save(ppt_path)
    print(f"PPT已保存至 {ppt_path}")

def pdf_to_ppt(pdf_path, ppt_path, dpi=150):
    """从PDF文件生成PPT文件"""
    images = pdf_to_images(pdf_path, dpi)
    images_to_ppt(images, ppt_path)

# 使用示例
pdf_path = './开题.pdf'  # 输入PDF文件路径
ppt_path = './2021251124古翱翔开题.pptx'    # 输出PPT文件路径
dpi = 600  # 设置较高的dpi值以确保图像清晰度

pdf_to_ppt(pdf_path, ppt_path, dpi)


PPT已保存至 ./2021251124古翱翔开题.pptx
